In [1]:
import os
import gc
os.chdir('../../')

In [27]:
import pandas as pd
import numpy as np
import json

### Read the parquet file located at `./week_1/data/processed/full_churn_data_with_target.parquet`

In [3]:
df = pd.read_parquet('./week_1/data/processed/full_churn_data_with_target.parquet')
df.reset_index(drop=False, inplace=True) 
df.sort_values(by=['customer_id', 'date'], ascending=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5286530 entries, 0 to 5286529
Data columns (total 31 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   Id                        int64         
 1   customer_id               int64         
 2   interest_rate             float64       
 3   name                      object        
 4   country                   object        
 5   date_of_birth             object        
 6   address                   object        
 7   date                      datetime64[ns]
 8   atm_transfer_in           int64         
 9   atm_transfer_out          int64         
 10  bank_transfer_in          int64         
 11  bank_transfer_out         int64         
 12  crypto_in                 int64         
 13  crypto_out                int64         
 14  bank_transfer_in_volume   float64       
 15  bank_transfer_out_volume  float64       
 16  crypto_in_volume          float64       
 17  crypto_o

### Feature Engineering

In [4]:
# Touchpoints nested list to individual count columns

# 1. Explode the touchpoints list column to transform each channel into its own row
exploded_df = df.explode('touchpoints')

# 2. Count the occurrences of each channel per original row
counts = (exploded_df.groupby([exploded_df.index, 'touchpoints'])
                     .size()
                     .unstack(fill_value=0))

exploded_df = None

# 3. Ensure all possible columns exist (email, appointment, phone, whatsapp)
for col in ['email', 'appointment', 'phone', 'whatsapp']:
    if col not in counts.columns:
        counts[col] = 0

# 4. Merge these counts back into the original DataFrame
df = df.join(counts, how='left').fillna(0)
counts = None

# 5. Convert counts to integer type
df[['email', 'appointment', 'phone', 'whatsapp']] = (
    df[['email', 'appointment', 'phone', 'whatsapp']].astype(int)
)

gc.collect()

0

In [5]:
# Base churn
df['days_between'] = df.groupby('customer_id')['date'].diff().dt.days.fillna(0)
df['customer_age'] = (df['date'] - pd.to_datetime(df['date_of_birth'])).dt.days / 365.25
df['from_competitor'] = df['from_competitor'].astype(int)
df['churn_due_to_fraud'] = df['churn_due_to_fraud'].astype(int)
df['atm_transfer_in'] = df['atm_transfer_in']
df['atm_transfer_out'] = df['atm_transfer_out']

In [6]:
# Window features
print('Lifetime Window features...')
df['prior_emails'] = df.groupby('customer_id')['email'].cumsum().values
df['prior_appointments'] = df.groupby('customer_id')['appointment'].cumsum().values
df['prior_phones'] = df.groupby('customer_id')['phone'].cumsum().values
df['prior_whatsapps'] = df.groupby('customer_id')['whatsapp'].cumsum().values

df['prior_count'] = df.groupby('customer_id').cumcount() + 1
df['prior_bank_balance'] = df.groupby('customer_id')['bank_transfer_in_volume'].cumsum().values - df.groupby('customer_id')['bank_transfer_out_volume'].cumsum().values
df['prior_crypto_balance'] = df.groupby('customer_id')['crypto_in_volume'].cumsum().values - df.groupby('customer_id')['crypto_out_volume'].cumsum().values
df['prior_mean_days_between'] = df.groupby('customer_id')['days_between'].expanding().mean().reset_index(level=0, drop=True)
df['prior_min_days_between'] = df.groupby('customer_id')['days_between'].expanding().min().reset_index(level=0, drop=True)
df['prior_max_days_between'] = df.groupby('customer_id')['days_between'].expanding().max().reset_index(level=0, drop=True)
df['prior_mean_bank_transfer_in'] = df.groupby('customer_id')['bank_transfer_in'].expanding().mean().reset_index(level=0, drop=True)
df['prior_mean_bank_transfer_out'] = df.groupby('customer_id')['bank_transfer_out'].expanding().mean().reset_index(level=0, drop=True)
df['prior_mean_crypto_in'] = df.groupby('customer_id')['crypto_in'].expanding().mean().reset_index(level=0, drop=True)
df['prior_mean_crypto_out'] = df.groupby('customer_id')['crypto_out'].expanding().mean().reset_index(level=0, drop=True)
df['prior_mean_bank_transfer_in_volume'] = df.groupby('customer_id')['bank_transfer_in_volume'].expanding().mean().reset_index(level=0, drop=True)
df['prior_mean_bank_transfer_out_volume'] = df.groupby('customer_id')['bank_transfer_out_volume'].expanding().mean().reset_index(level=0, drop=True)
df['prior_mean_crypto_in_volume'] = df.groupby('customer_id')['crypto_in_volume'].expanding().mean().reset_index(level=0, drop=True)
df['prior_mean_crypto_out_volume'] = df.groupby('customer_id')['crypto_out_volume'].expanding().mean().reset_index(level=0, drop=True)

def add_window_features(df, window_size=''):
    print('Adding window features for', window_size, '...')
    df[f'prior_{window_size}_count'] = df.groupby('customer_id').rolling(window_size, on='date')['date'].count().values
    df[f'prior_{window_size}_mean_days_between'] = df.groupby('customer_id').rolling(window_size, on='date')['days_between'].mean().values
    df[f'prior_{window_size}_max_days_between'] = df.groupby('customer_id').rolling(window_size, on='date')['days_between'].max().values
    df[f'prior_{window_size}_min_days_between'] = df.groupby('customer_id').rolling(window_size, on='date')['days_between'].min().values
    df[f'prior_{window_size}_mean_bank_transfer_in'] = df.groupby('customer_id').rolling(window_size, on='date')['bank_transfer_in'].mean().values
    df[f'prior_{window_size}_mean_bank_transfer_out'] = df.groupby('customer_id').rolling(window_size, on='date')['bank_transfer_out'].mean().values
    df[f'prior_{window_size}_mean_crypto_in'] = df.groupby('customer_id').rolling(window_size, on='date')['crypto_in'].mean().values
    df[f'prior_{window_size}_mean_crypto_out'] = df.groupby('customer_id').rolling(window_size, on='date')['crypto_out'].mean().values
    df[f'prior_{window_size}_mean_bank_transfer_in_volume'] = df.groupby('customer_id').rolling(window_size, on='date')['bank_transfer_in_volume'].mean().values
    df[f'prior_{window_size}_mean_bank_transfer_out_volume'] = df.groupby('customer_id').rolling(window_size, on='date')['bank_transfer_out_volume'].mean().values
    df[f'prior_{window_size}_mean_crypto_in_volume'] = df.groupby('customer_id').rolling(window_size, on='date')['crypto_in_volume'].mean().values
    df[f'prior_{window_size}_mean_crypto_out_volume'] = df.groupby('customer_id').rolling(window_size, on='date')['crypto_out_volume'].mean().values

# Last 10 days
add_window_features(df, '10D')

# Last 90 days
add_window_features(df, '90D')

# Last 180 days
add_window_features(df, '180D')

# Last 365 days
add_window_features(df, '365D')

# Last 365 days
add_window_features(df, '450D')

# This week volume
df.sort_values(by=['country', 'date'], ascending=True, inplace=True)
df['this_week_bank_volume'] = df.groupby('customer_id').rolling('7D', on='date')['bank_transfer_in_volume'].sum().values - \
                         df.groupby('customer_id').rolling('7D', on='date')['bank_transfer_out_volume'].sum().values

df['this_week_crypto_volume'] = df.groupby('customer_id').rolling('7D', on='date')['crypto_in_volume'].sum().values - \
                         df.groupby('customer_id').rolling('7D', on='date')['crypto_out_volume'].sum().values

#df['this_week_customer_count_by_country'] = df.groupby('country').rolling('7D', on='date')['customer_id'].nunique().values

Lifetime Window features...
Adding window features for 10D ...
Adding window features for 90D ...
Adding window features for 180D ...
Adding window features for 365D ...
Adding window features for 450D ...


### Defining diferent targets

In [10]:
# 365 days of inactivity
df['churn_365'] = 0
df.loc[df['days_diff'] >= 365, 'churn_365'] = 1

#420 days of inactivity
df['churn_420'] = 0
df.loc[df['days_diff'] >= 420, 'churn_420'] = 1

In [14]:
#All customers with a last activity date over a 18 months ago (reference date 2023-12-31).

'''
Verify this:

We're never very confident in our prediction at the point of churn.

We're asking a very hard question - is this transaction the last? Rather than asking has this customer churned after 100 days of inactivity have passed.

(Of the 2270378 in the train data, only 9820 are "final transactions", ~0.43%)
'''

churned_customers = df[df.date < '2024-01-01'].groupby('customer_id')['date'].max().reset_index()
churned_customers = churned_customers[churned_customers.date < '2022-06-01']
churned_customers.columns = ['customer_id', 'churn_date']
churned_customers['churn_18_months'] = 1

df = pd.merge(df, churned_customers, how='left', left_on=['customer_id', 'date'], right_on=['customer_id', 'churn_date'])
df.churn_18_months.fillna(0, inplace=True)

/var/folders/02/kk2lyyc1509_l0ltgwm_19y40000gq/T/ipykernel_11068/3564919720.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.churn_18_months.fillna(0, inplace=True)


### Saving the processed dataframe

In [26]:
non_features_list = ['Id', 'customer_id','name','date_of_birth','address','date','touchpoints','csat_scores','Usage','churn','next_date','days_diff','split','churn_365','churn_420','churn_date','churn_18_months']

In [ ]:
features_list = list(df.drop(columns=non_features_list).columns)
with open('./week_1/data/processed/final_features_list.json', 'w') as f:
    json.dump(features_list, f)

In [15]:
df.sort_values(by=['Id'], ascending=True, inplace=True)
df.Usage.replace(0, 'Public', inplace=True)
df.to_parquet('./week_1/data/processed/feature_engineering_dataset.parquet')

/var/folders/02/kk2lyyc1509_l0ltgwm_19y40000gq/T/ipykernel_11068/2335050495.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.Usage.replace(0, 'Public', inplace=True)
